In [148]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

In [149]:
df = pd.read_csv('futers.csv', index_col = 'moment')
df = df.fillna(0)
df.head()

,fiz_clients_in_long,fiz_clients_in_short,fiz_short_position,fiz_long_position,fiz_change_prev_week_short_abs,fiz_change_prev_week_long_abs,fiz_change_prev_week_short_perc,fiz_change_prev_week_long_perc,yur_clients_in_long,yur_clients_in_short,yur_short_position,yur_long_position,yur_change_prev_week_short_abs,yur_change_prev_week_long_abs,yur_change_prev_week_short_perc,yur_change_prev_week_long_perc,mos_index,mos_index_diff
moment,,,,,,,,,,,,,,,,,,
2012-10-02,92.0,74.0,708.0,1059.0,37.0,-240.0,0.05514,-0.18476,41.0,12.0,4441.0,4090.0,-9.0,268.0,-0.00202,0.07012,1483.03,-0.003186
2012-10-03,84.0,61.0,611.0,978.0,-97.0,-81.0,-0.13701,-0.07649,37.0,14.0,4519.0,4152.0,78.0,62.0,0.01756,0.01516,1471.61,-0.007700
2012-10-04,99.0,58.0,547.0,1039.0,-64.0,61.0,-0.10475,0.06237,36.0,15.0,4458.0,3966.0,-61.0,-186.0,-0.01350,-0.04480,1462.02,-0.006517
2012-10-05,99.0,73.0,901.0,1054.0,354.0,15.0,0.64717,0.01444,34.0,17.0,4504.0,4351.0,46.0,385.0,0.01032,0.09708,1475.60,0.009289
2012-10-08,95.0,70.0,621.0,1061.0,-280.0,7.0,-0.31077,0.00664,36.0,14.0,4903.0,4463.0,399.0,112.0,0.08859,0.02574,1474.69,-0.000617


In [150]:
target = (df['mos_index_diff'] > 0).astype(int)
X = df.iloc[:-1]
X_train = X
X_train = X_train.drop(['mos_index_diff'], axis = 1)
X_train['fiz_clients_in_long'] = X_train['fiz_clients_in_long']/(X['fiz_clients_in_long'] + X['fiz_clients_in_short'])
X_train['fiz_clients_in_short'] = X_train['fiz_clients_in_short']/(X['fiz_clients_in_long'] + X['fiz_clients_in_short'])
X_train['fiz_short_position'] = X_train['fiz_short_position']/(X['fiz_short_position'] + X['fiz_long_position'])
X_train['fiz_long_position'] = X_train['fiz_long_position']/(X['fiz_short_position'] + X['fiz_long_position'])
X_train['yur_clients_in_long'] = X_train['yur_clients_in_long']/(X['yur_clients_in_long'] + X['yur_clients_in_short'])
X_train['yur_clients_in_short'] = X_train['yur_clients_in_short']/(X['yur_clients_in_long'] + X['yur_clients_in_short'])
X_train['yur_short_position'] = X_train['yur_short_position']/(X['yur_short_position'] + X['yur_long_position'])
X_train['yur_long_position'] = X_train['yur_long_position']/(X['yur_short_position'] + X['yur_long_position'])
X_train = X_train.drop(['fiz_clients_in_short', 'fiz_short_position','yur_clients_in_short','yur_short_position',\
                        'fiz_change_prev_week_short_abs','yur_change_prev_week_long_abs',\
                       'fiz_change_prev_week_long_abs','yur_change_prev_week_short_abs' ], axis = 1)

y = target[1:]

In [151]:
print(y[:5])
X_train.head()

moment
2012-10-03    0
2012-10-04    0
2012-10-05    1
2012-10-08    0
2012-10-09    0
Name: mos_index_diff, dtype: int32


,fiz_clients_in_long,fiz_long_position,fiz_change_prev_week_short_perc,fiz_change_prev_week_long_perc,yur_clients_in_long,yur_long_position,yur_change_prev_week_short_perc,yur_change_prev_week_long_perc,mos_index
moment,,,,,,,,,
2012-10-02,0.554217,0.599321,0.05514,-0.18476,0.773585,0.479428,-0.00202,0.07012,1483.03
2012-10-03,0.579310,0.615481,-0.13701,-0.07649,0.725490,0.478838,0.01756,0.01516,1471.61
2012-10-04,0.630573,0.655107,-0.10475,0.06237,0.705882,0.470798,-0.01350,-0.04480,1462.02
2012-10-05,0.575581,0.539130,0.64717,0.01444,0.666667,0.491361,0.01032,0.09708,1475.60
2012-10-08,0.575758,0.630797,-0.31077,0.00664,0.720000,0.476511,0.08859,0.02574,1474.69


In [152]:
poly = PolynomialFeatures(2)
X_train = poly.fit_transform(X_train)

In [153]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
tscv = TimeSeriesSplit(n_splits=5)
parameters = {'C':np.linspace(0.1, 20, num=40)}

In [154]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
clf = LogisticRegression(random_state=0, solver='liblinear')

In [155]:
clf = GridSearchCV(clf, parameters, cv=tscv, scoring='roc_auc')

In [156]:
clf.fit(X_train_scaled,y)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='wa...
        7.75384615,  8.26410256,  8.77435897,  9.28461538,  9.79487179,
       10.30512821, 10.81538462, 11.32564103, 11.83589744, 12.34615385,
       12.85641026, 13.36666667, 13.87692308, 14.38717949, 14.8974359 ,
       15.40769231, 15.91794872, 16.42820513, 16.93846154, 17.44871795,
       

In [157]:
clf.best_score_

0.51932476852445